# Introduzione

L'obiettivo è quello di analizzare il dataset sui funghi, e testare diversi classificatori per trovare il migliore nel classificare nuovi funghi come commestibili o velenosi. Il documento è composto come segue:

- Data Exploration
    - Analisi classi dataset
    - Feature Selection
    - Rimozione righe malformate
- Test Classificatori
    - Naive Bayes
    - Random Forest
    - ...
    - ...

Import delle librerie necessarie

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
#altri import futuri

Caricamento del dataset

In [ ]:
df = pd.read_csv("dataset/secondary_data.csv", sep=";")
print("Dimension:", df.shape)
df.head()

# Data Exploration

Visualizzazione campi non nulli per ogni colonna/feature del dataset. Questo ci serve per vedere eventuali colonne che possono poi essere rimosse

In [ ]:
df.info()

## Class Balancing
Siamo interessati a classificare i funghi come velenosi o meno, per questo motivo identifichiamo le due classi di interesse a partire dalla colonna "class" del dataset. Vogliamo vedere quanto sono bilanciate le classi del dataset

In [ ]:
classes = [{"Commestibili": (df["class"] == "e").sum(), "Velenosi": (df["class"] == "p").sum()}]
total  = pd.DataFrame(classes)
total_e = float(total["Commestibili"])
total_p = float(total["Velenosi"])
mushrooms = [total_e, total_p]
mushrooms_labels = 'Edible','Poisonous'
fig, ax = plt.subplots(figsize = (8,6))
plt.pie(mushrooms,labels=mushrooms,autopct='%1.1f%%',colors = ['#88d14f', '#DA70D6'])
plt.title('Dataset Balancing', loc = "center", fontsize="20")
plt.axis('equal')
plt.legend(mushrooms_labels,title="Total Mushrooms",bbox_to_anchor=(0.6, -0.05, 0, 0))
fig.set_facecolor('white')
plt.show()

Vediamo che ci sono in percetuale il 10% in più di funghi velenosi, e questo può portare il classificatore a tendere nel classificare funghi come velenosi (falso positivo). Questo errore è meno grave del falso negativo (perche se ti dico che un fungo è velenoso comunque non lo mangi, il contrario invece è velenoso ti dico di no e muori). 

Possiamo pensare di ribilanciare le classi tramite oversampling o undersampling, oppure ancora meglio con SMOTE.

Rivalutare magari il class balancing dopo rimozione delle righe con valori nulli.

## Conversione Features e Mapping

Conversione delle features categoriche in nominali, si modifica prima il tipo di dato dei valori delle features da 'object' in 'category' per poi procedere con l'effettiva trasformazione dei valori categorici.

In [ ]:
df = df.astype({"class": 'category', "cap-shape": 'category', "cap-surface": 'category', "cap-color": 'category', "does-bruise-or-bleed": 'category', 
"gill-attachment": 'category', "gill-spacing": 'category', "gill-color": 'category', "stem-surface": 'category', "stem-color": 'category',  "has-ring": 'category', 
"ring-type": 'category', "habitat": 'category', "season": 'category'}, errors = 'raise')
df.dtypes

Si trasformano i valori categorici in numerici (int64) e si salva il dizionario che mantiene il mapping per un eventuale utilizzo futuro.

In [ ]:
mapping = []
encoder = LabelEncoder()
for i in range(len(df.columns)):
    value = df[df.columns[i]] 
    if (value.dtype == "category"):
        print(value)
        df[df.columns[i]] = encoder.fit_transform(value)
        mapping_dict = {index : label for index , label in enumerate(encoder.classes_)}
        mapping.append(mapping_dict)

mapping

## Feature Visualization

Visualizziamo il numero di classi per ogni feature in modo da visualizzare possibili colonne da rimuovere perche con valori costanti o quasi.

In [ ]:
categorical = ['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color', 'stem-root', 'stem-surface', 
'stem-color', 'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color', 'habitat', 'season']
continous = ['cap-diameter', 'stem-height', 'stem-width']

fig = plt.figure(figsize=(25,20))
for i in range(0, len(categorical)) :
    fig.add_subplot(5,4,i+1)
    sns.countplot(x=categorical[i], data=df, alpha=.7)

for col in continous:
    fig.add_subplot(5,4,i+2)
    i+=1
    sns.histplot(df[col].dropna(), kde_kws={"lw": 2, "color": 'xkcd:cadet blue'})

## Feature Selection

Si selezionano le feature più utili ai fini della classificazione

### Rimozione di features con dati costanti

Primo approccio base di feature selection andiamo a rimuovere le feature con varianza nulla presenti nel df, ovvero quelle features il cui valore è costante.   
Dalla fase di Data Visualization abbiamo notato che una features con varianza nulla è il veil-type.
 
TODO Valutare l'utilizzo di una VarianceThreshold from SkLearn per rimuovere le features con varianza bassa

In [ ]:
df = df.drop("veil-type", 1)
df.isnull().sum()

### Rimozione di features con valori nulli

Si cancellano le features che presentano più del 50% di valori nulli. Per le colonne con meno del 50% di valori nulli si utilizza invece un meccanismo di imputazione per sopperire alla presenza di dati mancanti in quelle features da scartare ma che hanno buona correlazione (vd. dopo) con la variabile target e buona distribuzione dei valori.

In [43]:
# TODO cap-surface,14120 | gill-attachment,9884 | gill-spacing,25063 | stem-surface,38124 | ring-type,2471
df = df.drop("veil-color", 1)
df = df.drop("spore-print-color", 1)
df = df.drop("stem-root", 1)
df.head()

C:\Users\danil\AppData\Local\Temp\ipykernel_12696\1379450327.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop("veil-color", 1)
C:\Users\danil\AppData\Local\Temp\ipykernel_12696\1379450327.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop("spore-print-color", 1)
C:\Users\danil\AppData\Local\Temp\ipykernel_12696\1379450327.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop("stem-root", 1)


,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-surface,stem-color,veil-type,has-ring,ring-type,habitat,season
0,p,15.26,x,g,o,f,e,NaN,w,16.95,17.09,y,w,u,t,g,d,w
1,p,16.60,x,g,o,f,e,NaN,w,17.99,18.19,y,w,u,t,g,d,u
2,p,14.07,x,g,o,f,e,NaN,w,17.80,17.74,y,w,u,t,g,d,w
3,p,14.17,f,h,e,f,e,NaN,w,15.77,15.98,y,w,u,t,p,d,w
4,p,14.64,x,h,o,f,e,NaN,w,16.53,17.20,y,w,u,t,p,d,w


### Imputazione delle features con valori nulli

Si utilizza un meccanismo di imputazione per sopperire alla presenza di dati mancanti per le features da scartare per la politica scelta (nan < 50%), ma che presentano una buona correlazione con la variabile target e una buona distribuzione dei valori.
Ovviamente il meccanismo è applicato anche a quelle features con meno del 20% di valori nulli, così da ripristinare la variabile per l'uso nella classificazione rimpiazzando tutti i dati mancanti.

In [ ]:
plt.figure(figsize=(16,12))

sns.heatmap(dataset.corr(), linewidths=.1, cmap=sns.diverging_palette(220, 10, as_cmap=True), annot=True, annot_kws={"size": 8})

plt.yticks(rotation=0);
#plt.savefig("corr.png", format='png', dpi=400, bbox_inches='tight')